In [37]:
from transformers import BertConfig, BertModel, AutoTokenizer
from sklearn.model_selection import train_test_split
import torch
import pandas as pd
from datasets import load_dataset, Dataset, DatasetDict

In [38]:
tokenizer = AutoTokenizer.from_pretrained("TurkuNLP/bert-base-finnish-cased-v1") # testataan valmiiksi koulutettua tokenizeria

In [39]:
# Loading dataset
dataSetPath = "2011-2018-3.csv"

df = pd.read_csv(dataSetPath, sep="␞", nrows=10000)
print(df.columns)

C:\Users\joakim\AppData\Local\Temp\ipykernel_10516\1848190111.py:4: ParserWarning: Falling back to the 'python' engine because the separator encoded in utf-8 is > 1 char long, and the 'c' engine does not support such separators; you can avoid this warning by specifying engine='python'.
  df = pd.read_csv(dataSetPath, sep="␞", nrows=10000)


Index(['id', 'url', 'headline', 'text', 'subjects', 'datePublished'], dtype='object')


In [40]:
dataset = Dataset.from_pandas(df)
dataset

Dataset({
    features: ['id', 'url', 'headline', 'text', 'subjects', 'datePublished'],
    num_rows: 10000
})

In [41]:

train_test_split = dataset.train_test_split(test_size=0.2)

train_valid_split = train_test_split['train'].train_test_split(test_size=0.1)

datasets = DatasetDict({
    'train': train_valid_split['train'],
    'validation': train_valid_split['test'],
    'test': train_test_split['test']
})
print(datasets)

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'headline', 'text', 'subjects', 'datePublished'],
        num_rows: 7200
    })
    validation: Dataset({
        features: ['id', 'url', 'headline', 'text', 'subjects', 'datePublished'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'url', 'headline', 'text', 'subjects', 'datePublished'],
        num_rows: 2000
    })
})


In [42]:
unique_values = set(datasets["train"]["subjects"])
print(unique_values)


{"['kulttuuri']", "['kotimaa', 'talous']", "['ulkomaat', 'kulttuuri', 'tiede']", "['kulttuuri', 'urheilu']", "['ulkomaat', 'luonto', 'sää']", "['ulkomaat', 'politiikka']", "['luonto', 'kotimaa', 'sää']", "['kotimaa', 'talous', 'tiede']", "['luonto', 'ulkomaat', 'tiede']", "['oppiminen']", "['urheilu', 'kulttuuri']", "['kotimaa', 'oppiminen', 'kolumnit']", "['kulttuuri', 'tiede']", "['kulttuuri', 'kotimaa']", "['tiede', 'ulkomaat']", "['sää', 'luonto']", "['luonto', 'ulkomaat']", "['luonto']", "['ulkomaat', 'urheilu']", "['luonto', 'talous']", "['talous', 'kotimaa', 'politiikka']", "['kotimaa', 'politiikka']", "['politiikka']", "['kotimaa', 'ulkomaat', 'tiede']", "['kotimaa', 'oppiminen']", "['urheilu', 'luonto']", "['kotimaa', 'sää', 'luonto']", "['politiikka', 'talous']", "['sää']", "['tiede', 'talous']", "['kulttuuri', 'ulkomaat']", "['kotimaa', 'ulkomaat', 'tiede', 'luonto', 'sää']", "['kotimaa', 'tiede']", "['politiikka', 'tiede']", "['kotimaa', 'kulttuuri']", "['ulkomaat', 'kotima

In [49]:
def tokenize_function(example):
    return tokenizer(text_target=example["text"], truncation=True)

In [53]:
# tokenizer(text_target=example["text"], truncation=True)
# tokenizer(datasets["train"]["text"], truncation=True)
tokenized_datasets = datasets.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/7200 [00:00<?, ? examples/s]

['\n Vuoden 2010 kesä tullaan muistamaan pitkään, niin uusien lämpötilaennätysten kuin myös äärevien sääilmiöiden osalta. Rajuilmojen yhteydessä esimerkiksi osa Suomen puustosta kärsi mittavia vahinkoja. (Artikkeli julkaistu alunperin lokakuussa 2010.)\n Monessa kohtaa puhuttiin, että myrskyt olivat kesällä vahingonaiheuttajia. Ilmaisu on hieman virheellinen, sillä myrskylle on olemassa tarkka meteorologinen määritelmä, joka ei kesän rajuilmojen yhteydessä täyttynyt yhdessäkään.\n Myrskystä voidaan puhua tilanteessa, jolloin kymmenen minuutin keskituuli on vähintään 21 metriä sekunnissa. Kansainvälisen määritelmän mukaan tuulen tulee puhaltaa vähintään 25 metriä sekunnissa. Lisää hämmennystä aiheuttaa se, että sana storm tarkoittaa englannin kielessä myrskytuulen lisäksi myös voimakasta sääilmiötä. Tästä syystä ulkomailla uutisoitu voimakas sääilmiö, esimerkiksi "thunder" tai "snowstorm", muuttuu Suomen kielelle käännettäessa usein myrskyksi, vaikka parempia termejä olisivat esimerkiks

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]